In [2]:
from pyspark.sql import Window
from pyspark.sql.window import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALSModel

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local") \
    .getOrCreate()

In [ ]:
business = spark.read.json("../../temp/yelp_json_yelp_academic_dataset_business.json")
review = spark.read.json("../../temp/yelp_json_yelp_academic_dataset_review.json")
user = spark.read.json("../../temp/yelp_json_yelp_academic_dataset_user.json")

In [17]:
userRatings=review.groupBy("user_id").count()
window = Window.orderBy(col('user_id'))
userRatings = userRatings.withColumn('userid', row_number().over(window))
buiRatings=review.groupBy("business_id").count()
window = Window.orderBy(col('business_id'))
buiRatings = buiRatings.withColumn('businessid', row_number().over(window))
newratings=review.join(userRatings, ['user_id'])
newratings=newratings.join(buiRatings, ['business_id'])
newratings=newratings.withColumn("stars",col("stars").cast(IntegerType()))
newratings=newratings.withColumn("date",to_timestamp("date"))

In [3]:
model=ALSModel.load("..\data\save")

In [18]:
userid = [ [2]]

useridColumns = StructType([StructField("userid", IntegerType())])
deptDF = spark.createDataFrame(data=userid, schema = useridColumns)

userSubsetRecs = model.recommendForUserSubset(deptDF, 1000)
nrecommendations = userSubsetRecs\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userid', col("rec_exp.businessid"), col("rec_exp.rating"))

nrecommendations.show()

D:\spark\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+----------+---------+
|userid|businessid|   rating|
+------+----------+---------+
|     2|     35182|7.3350387|
|     2|    140227|7.2798605|
|     2|     62843| 7.273821|
|     2|     46174|7.1696444|
|     2|    102258| 6.989647|
|     2|     73731| 6.934354|
|     2|     81355|6.8198047|
|     2|     82077|6.8111367|
|     2|     72090|6.7951193|
|     2|    111232|6.7868247|
|     2|     36492|6.7769537|
|     2|    128580|6.7675514|
|     2|     38648| 6.743578|
|     2|     48202|6.7410393|
|     2|     77870|6.7246947|
|     2|     17069| 6.716116|
|     2|     43128|6.7052073|
|     2|      9574| 6.693309|
|     2|    106583| 6.660489|
|     2|     20597| 6.645897|
+------+----------+---------+
only showing top 20 rows



In [19]:
import pickle
with open('../data/res_scores.pickle', 'rb') as handle:
    res = pickle.load(handle)
nrecommendations=nrecommendations.filter(col('businessid').isin(list(res["businessid"])))

In [21]:
nrecommendations=nrecommendations.limit(10)
lst = nrecommendations.select('businessid').rdd.flatMap(lambda x: x).collect()
col1_list = list(lst)
lstres = res[res['businessid'].isin(col1_list)]
lstres[["business_id", "name", "score"]]

,business_id,name,score
5766,FYQI6L-Ty_k2O-UP_T1DAA,Elevated Wellness,4.781041
1852,f4rSatoGZoW2p_4OWrDZ1A,Tempus,4.723525
18902,2HDRsM0c-k8KAmKiMaDw6A,Captain Andy's Market,4.638445
10869,qCgaKAj-CbU0L1EzaJrQug,Hop House Tennessee Taps,4.628672
777,fsGytCX6xLYIM4oniPP-og,Louisiana's Soul Bayou,4.602937
22393,XD32ijW7RqkVeaTvOty_Kw,Cardinal Landscaping,4.503389
15409,yhMpYBlu_yimn9sT-cIayA,Fitz and Starts,4.474044
5406,vRVoNDTLQl0lF212AjXepA,Made by Lia,4.471835
8991,1Sl5-YrdHQNwd_LuKbxQ7A,Copper Shaker,4.403525
1784,_uQEKHH0R5fxrYkLrUI9pQ,Gu Wei Noodles & Grill,4.347151


In [13]:
nrecommendations.show()

+------+----------+---------+
|userid|businessid|   rating|
+------+----------+---------+
|     2|    100920|6.3240767|
|     2|     80273|6.2439375|
|     2|    127192|6.1891665|
|     2|    102835|6.1520953|
|     2|    147315|6.0846972|
|     2|    139478|6.0144596|
|     2|      7844|6.0132546|
|     2|     88919|   6.0092|
|     2|      5946|5.9493423|
|     2|     39082|5.9452047|
|     2|     92121|5.9278507|
|     2|     88077|5.9215956|
|     2|    146139|5.9161215|
|     2|     55870| 5.898738|
|     2|     72088| 5.893149|
|     2|     61334| 5.885145|
|     2|     31964| 5.872604|
|     2|     17936|5.8648252|
|     2|     58314| 5.856209|
|     2|    132934|5.8504457|
+------+----------+---------+
only showing top 20 rows

